In [3]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [5]:
STORM_NUMBER = 9
WINTER = "2024-2025"
TITLE = "21-22 January 2025"
SUBTITLE = "8 AM 23 January 2025"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2025, 1, 22, 20)
ets = datetime(2025, 1, 23, 18)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)

In [6]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [7]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [8]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,MI,APX,19.0,-86.240000,44.220000,POINT (1090016.206 5950.548),True,0,True,LSR,27,18
1,MI,GRR,18.5,-86.110000,43.250000,POINT (1118329.334 -98995.518),True,1,True,LSR,28,14
2,MI,GRR,18.0,-86.350000,43.500000,POINT (1094661.227 -74753.497),True,2,True,LSR,27,15
3,MT,TFX,17.0,-110.700000,46.840000,POINT (-810947.458 258721.318),True,3,True,LSR,-35,26
4,MI,APX,15.0,-86.110000,44.490000,POINT (1095098.027 37404.449),True,4,True,LSR,27,19
...,...,...,...,...,...,...,...,...,...,...,...,...
2876,IA,DVN,0.0,-91.033297,41.449973,POINT (745844.281 -353970.512),False,IA-MC-20,False,COCORAHS,16,6
2877,IA,DVN,0.0,-91.563410,40.841769,POINT (708625.883 -425986.382),False,IA-HY-5,False,COCORAHS,14,3
2878,MN,MPX,0.0,-93.486477,44.063137,POINT (519765.234 -83345.379),False,MN-WC-12,False,COCORAHS,8,15
2879,NE,OAX,0.0,-96.037890,41.132019,POINT (331863.07 -422038.143),False,NE-SY-36,False,COCORAHS,2,4


In [9]:
def main():
    setpoints = [

    ]
    #if setpoints:
    #    add_setpoints(setpoints)
    cull = [
        'Z19',
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0., upper=2)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250123.png")
    mp.close()


main()

     state  wfo  val        lon        lat                             geo  \
79      IA  DVN  4.0 -91.690000  42.040000  POINT (685026.168 -294366.453)   
89      IA  DVN  3.5 -91.680000  41.710000   POINT (689456.618 -330831.38)   
90      IA  DVN  3.5 -91.640000  42.020000  POINT (689350.435 -296162.409)   
97      IA  DVN  3.3 -91.710000  41.980000  POINT (684039.643 -301178.851)   
98      IA  DVN  3.3 -91.110000  42.080000   POINT (732143.686 -284922.99)   
...    ...  ...  ...        ...        ...                             ...   
2859    IA  DMX  0.0 -94.227390  43.205520  POINT (467443.182 -182994.525)   
2874    IA  ARX  0.0 -92.717640  43.283720  POINT (588571.351 -164548.533)   
2875    IA  DMX  0.0 -93.824195  42.452984  POINT (506209.946 -264096.737)   
2876    IA  DVN  0.0 -91.033297  41.449973  POINT (745844.281 -353970.512)   
2877    IA  DVN  0.0 -91.563410  40.841769  POINT (708625.883 -425986.382)   

      used_for_analysis     nwsli  plotme    source  xcell  yce

## 